In [ ]:
import os
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, LLMConfig
from crawl4ai.extraction_strategy import LLMExtractionStrategy
from crawl4ai.content_filter_strategy import BM25ContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from pydantic import BaseModel
from typing import List, Optional


class CrawlerTaskConfig(BaseModel):
    task_name: str
    urls: List[str]
    output_dir: str


class PageData(BaseModel):
    title: str
    content: str
    images: List[str]


class PageResult(BaseModel):
    url: str
    data: PageData

async def crawl(task_config: CrawlerTaskConfig,
                schema: BaseModel,
                browser_cfg: BrowserConfig = None,
                logfile: str = None):

    instruction = """
Extract the main article content from this web page.
Only include text that is directly related to the main title of the article:
    - The main title
    - The main content
    - The image URLs in the main content.
Do not include navigation menus, advertisements, sidebars, related articles, comments, or footer content.
Retain the content original language.
Return valid JSON.
""".strip()

    if task_config.task_name:
        instruction = f"This page is about {task_config.task_name}.\n" + instruction
        instruction.strip()

    # llm_strategy = LLMExtractionStrategy(
    #     llm_config=LLMConfig(provider=f"azure/{os.getenv("AOAI_DEPLOYMENT")}",
    #                          api_token=os.getenv('AZURE_OPENAI_API_KEY'),
    #                          base_url=os.getenv("AOAI_ENDPOINT")),
    #     extraction_type="schema",
    #     apply_chunking=False,
    #     instruction=instruction,
    #     schema=schema,
    #     input_format="html",
    #     extra_args={"temperature": 0.0, "max_tokens": 1500},
    #     verbose=True,
    # )

    bm25_filter = BM25ContentFilter(
        user_query=task_config.task_name,
        # Adjust for stricter or looser results
        bm25_threshold=1.2,
    )

    md_generator = DefaultMarkdownGenerator(
        content_filter=bm25_filter,
        options={
            "ignore_images": False
        })

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        wait_for="body",
        markdown_generator=md_generator,
        css_selector="body",
        # target_elements=["body", "img"],
        excluded_selector="script, style",
        # extraction_strategy=llm_strategy,
        # only_text=True,
        exclude_external_images=True,
        image_score_threshold=3,
        # exclude_all_images=True,
    )

    if not browser_cfg:
        browser_cfg = BrowserConfig(headless=True)

    from datetime import datetime
    now = datetime.now().strftime("%Y%m%d")
    file_name = f"{task_config.task_name.replace(" ", "-")}__{now}.md"
    output_file_path = os.path.join(task_config.output_dir, file_name)

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        

        for url in task_config.urls:
            
            
            with open(output_file_path, "a+") as file:
                file.write(url)
                file.write(result.markdown.fit_markdown)
                file.write("\n\n")

            if logfile:
                with open(logfile, "a+") as file:
                    file.write(f"Crawled {url} to {output_file_path}\n")
                    # file.write(f"LLM usage: {llm_strategy.usages}\n")
                    file.write("\n")

browser_cfg = BrowserConfig(headless=True)

In [20]:
from pydantic_core import from_json

with open("/Users/quanbm/Dev/sides/localgaid_notebooks/crawler_task/configs/dinhongnamhai.json", "r") as file:
    task_config = CrawlerTaskConfig.model_validate(from_json(file.read(), allow_partial=True))
print(repr(task_config))

CrawlerTaskConfig(task_name='Dinh Ông Nam Hải', urls=['https://vietnamtourism.gov.vn/post/39240', 'https://ittpa.baria-vungtau.gov.vn/article?item=d228276ba001e8f412067e8d4d22eeaf', 'https://baobariavungtau.com.vn/van-hoa-nghe-thuat/202503/le-hoi-nghinh-ong-nam-hai-thu-hut-hang-ngan-nguoi-tham-du-1036953/index.htm', 'https://www.huongphonghococbeachresort.com/y-nghia-le-hoi-nghinh-ong-tai-phuoc-hai-%E2%80%93-van-hoa-tam-linh-cua-nguoi-dan-bien.html'], output_dir='/Users/quanbm/Dev/sides/localgaid_notebooks/data/bronze')


In [24]:
await crawl(task_config=task_config,
            schema=PageData.model_json_schema(),
            browser_cfg=browser_cfg,
            logfile="/Users/quanbm/Dev/sides/localgaid_notebooks/crawler_task/logs/logs.txt")

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://vietnamtourism.gov.vn/post/39240                                                             |
✓ | ⏱: 2.55s 

[SCRAPE].. ◆ https://vietnamtourism.gov.vn/post/39240                                                             |
✓ | ⏱: 0.10s 

[COMPLETE] ● https://vietnamtourism.gov.vn/post/39240                                                             |
✓ | ⏱: 2.66s 

Images: [{'src': 'https://images.vietnamtourism.gov.vn/vn//images/2022/thang_1/brvt_namhai.jpg', 'data': '', 'alt': '', 'desc': 'Hình ảnh tại lễ hội Dinh Ông Nam Hải năm 2019. Ảnh: Sưu tầm Với nhiều tên gọi khác nhau như: Lễ cầu ngư; Lễ cúng Ông; Lễ rước cốt Ông; Lễ nghinh Ông Thủy Thần; Lễ Dinh Ông Nam Hải nhưng hầu hết các tên gọi đều gắn với tục thờ “Cá Ông” là sinh vật thiêng ở biển cả, là vị cứu tinh đối với người dân đánh cá nói riêng và người làm nghề mưu sinh trên biển nói chung”. Tục thờ “Cá Ông” đã trở thành một tín ngưỡng dân gian được truyền từ đời này sang đời khác trong vùng biển Phước Hải. Chính vì thế, Lễ hội Dinh Ông Nam Hải đã trở thành một nét văn hóa đặc trưng của người dân vùng biển và thu hút đông đảo du khách đến tham dự. Hình ảnh tại lễ hội Dinh Ông Nam Hải năm 2019. Ảnh: Sưu tầm Việc tổ chức lễ hội nhằm đáp ứng nhu cầu văn hóa tâm linh của người dân nói chung và ngư dân nói riêng. Đây cũng là dịp để ngư dân thể hiện lòng thành kính của bà con với thần Nam Hải đ

...

Images: [{'src': '/dataimages/202503/original/images2007038_dua_aaa.jpg', 'data': '', 'alt': '  Biểu diễn ghe mô hình trên biển tại lễ hội Nghinh Ông Nam Hải.', 'desc': 'Biểu diễn ghe mô hình trên biển tại lễ hội Nghinh Ông Nam Hải.', 'score': 5, 'type': 'image', 'group_id': 4, 'format': 'jpg', 'width': None}]


[FETCH]... ↓ https://www.huongphonghococbeachresort.com/y-ngh...c-hai-–-van-hoa-tam-linh-cua-nguoi-dan-bien.html  |
✓ | ⏱: 1.10s 

[SCRAPE].. ◆ https://www.huongphonghococbeachresort.com/y-ngh...c-hai-–-van-hoa-tam-linh-cua-nguoi-dan-bien.html  |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://www.huongphonghococbeachresort.com/y-ngh...c-hai-–-van-hoa-tam-linh-cua-nguoi-dan-bien.html  |
✓ | ⏱: 1.12s 

Images: [{'src': 'https://www.huongphonghococbeachresort.com/uploads/images1792816_nghinh__ng.jpg', 'data': '', 'alt': 'Ý Nghĩa Lễ Hội Nghinh Ông Tại Phước Hải – Văn Hóa Tâm Linh Của Người Dân Biển', 'desc': 'Ý Nghĩa Lễ Hội Nghinh Ông Tại Phước Hải – Văn Hóa Tâm Linh Của Người Dân Biển 18/12/2024 - Tin Tức Địa Phương Lễ hội Nghinh Ông tại Phước Hải, Bà Rịa - Vũng Tàu, không chỉ là một sự kiện văn hóa đặc sắc mà còn mang trong mình giá trị tâm linh sâu sắc. Đây là dịp để người dân biển bày tỏ lòng tri ân với Cá Ông (cá voi) – vị thần bảo hộ của ngư dân trong những chuyến hải trình đầy thử thách, đồng thời thể hiện sự đoàn kết, gắn bó cộng đồng và lòng biết ơn thiên nhiên. 1. Nguồn gốc và ý nghĩa tâm linh Theo tín ngưỡng dân gian, Cá Ông là loài linh thiêng, luôn giúp đỡ ngư dân vượt qua sóng dữ và những hiểm họa trên biển. Người dân Phước Hải tôn thờ Cá Ông như một vị thần hộ mệnh, gọi bằng những tên kính cẩn như "Ông Nam Hải" hoặc "Ông Lớn". Lễ hội Nghinh Ông, diễn ra vào tháng Hai âm 

In [5]:
result.media["images"]

NameError: name 'result' is not defined